# My Top Queries

# Exercise 3

\-- Write a query that returns customer and employee pairs

\-- that had order activity in January 2016 but not in February 2016

\-- Tables involved: TSQLV4 database, Orders table

## My Answer

<u>**Before ChatGPT**</u>

**Proposition:** Write a query that selects unique customer and employee pairs who placed orders in January 2016 but not in February 2016.

**Table:** Sales.\[Order\] aliased as O, LEFT JOIN <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Sales.[Order] aliased as SO</span>

**Columns:** OrderDate, CustomerId and EmployeeId

**Predicate:**

```
LEFT JOIN Sales.[Order] SO ON O.CustomerId = SO.CustomerId AND O.EmployeeId = SO.EmployeeId
    AND SO.OrderDate >= '2016-02-01' AND SO.OrderDate < '2016-03-01'

```

The join should happen between rows where both the CustomerId and EmployeeId match between the original orders table (O) and the self-joined orders table (SO) then refines the join to consider only those rows in SO that have an order date in February 2016. Effectively filtering the orders in the self-joined table to this specific month.

```
WHERE O.OrderDate >= '2016-01-01' AND O.OrderDate < '2016-02-01'
    AND OJ.OrderDate IS NULL

```

This WHERE clause, in combination with the LEFT JOIN, filters the Sales.\[Order\] table to only return rows where orders were placed in January 2016. It ensures that these selected orders have no matching orders in February 2016 for the same customer and employee pair.

**What's special:**

**LEFT JOIN:**

```
LEFT JOIN Sales.[Order] SO ON O.CustomerId = SO.CustomerId AND O.EmployeeId = SO.EmployeeId
    AND SO.OrderDate >= '2016-02-01' AND SO.OrderDate < '2016-03-01'

```

A self-join is used on the same orders table to compare orders across two different time periods (January and February of the same year).

**Filtering for January Orders:**

```
WHERE O.OrderDate >= '2016-01-01' AND O.OrderDate < '2016-02-01'

```

The WHERE clause specifically filters orders in the O table to those placed in January 2016.

**Ensuring No February Orders:**

```
 AND SO.OrderDate IS NULL;

```

The check OJ.OrderDate IS NULL after the LEFT JOIN ensures that we're selecting only those January orders for which there is no corresponding order in February 2016 for the same customer and employee. This is possible because, in the context of a LEFT JOIN, if there's no matching row in the SO table (representing February orders), the columns from OJ (including OrderDate) will be NULL.

**DISTINCT:** Ensures that each customer and employee pair is unique in the result set, eliminating any duplicate orders placed by the same customer with the same employee within January.

<br></br>

<u>**After ChatGPT**</u>

**Proposition:** Write a query that selects unique customer and employee pairs who placed orders in January 2016 but not in February 2016.

**Table:** Sales.\[Order\]

**Columns:** CustomerId and EmployeeId

**Predicate:**

For January Orders:

```
WHERE OrderDate >= '2016-01-01' AND OrderDate < '2016-02-01'

```

This WHERE clause filters the Sales.\[Order\] table to only return rows where the OrderDate is in January 2016.

For February Orders:

```
WHERE OrderDate >= '2016-02-01' AND OrderDate < '2016-03-01'

```

**What's special:**

**Common Table Expressions:**

The query uses two Common Table Expressions, "JanuaryOrders" and "FebruaryOrders". Each Common Table Expressions selects CustomerId and EmployeeId from the Sales.\[Order\] table filtered by the order dates for January and February 2016.

**JanuaryOrders CTE:** Identifies all orders placed in January 2016 by filtering orders where the OrderDate falls within January 2016.

**FebruaryOrders CTE:** Similarly, identifies all orders placed in February 2016.

**LEFT JOIN:** In the main query, a LEFT JOIN is performed between these two Common Table Expressions based on CustomerId and EmployeeId to find pairs present in January but not in February by checking for NULLs in the FebruaryOrders CTE (WHERE f.CustomerId IS NULL AND f.EmployeeId IS NULL).

In [ ]:
--Before using Chatgpt

USE TSQLV6;
SELECT DISTINCT O.custid, O.empid
FROM Sales.Orders O
LEFT JOIN Sales.Orders OJ ON O.custid = OJ.custid AND O.empid = OJ.empid 
    AND OJ.orderdate >= '2016-02-01' AND OJ.orderdate < '2016-03-01'
WHERE O.orderdate >= '2016-01-01' AND O.orderdate < '2016-02-01'
    AND OJ.orderid IS NULL;


USE Northwinds2022TSQLV7;
SELECT DISTINCT O.CustomerId, O.EmployeeId
FROM Sales.[Order] O 
LEFT JOIN Sales.[Order] SO ON O.CustomerId = SO.CustomerId AND O.EmployeeId = SO.EmployeeId
    AND SO.OrderDate >= '2016-02-01' AND SO.OrderDate < '2016-03-01'
WHERE O.OrderDate >= '2016-01-01' AND O.OrderDate < '2016-02-01'
    AND SO.OrderDate IS NULL;


--After using Chatgpt

USE TSQLV6;
WITH JanuaryOrders AS (
    SELECT custid, empid
    FROM Sales.Orders
    WHERE orderdate >= '2016-01-01' AND orderdate < '2016-02-01'
),
FebruaryOrders AS (
    SELECT custid, empid
    FROM Sales.Orders
    WHERE orderdate >= '2016-02-01' AND orderdate < '2016-03-01'
)
SELECT j.custid, j.empid
FROM JanuaryOrders j
LEFT JOIN FebruaryOrders f ON j.custid = f.custid AND j.empid = f.empid
WHERE f.custid IS NULL AND f.empid IS NULL
ORDER BY custid;


USE Northwinds2022TSQLV7;
WITH JanuaryOrders AS (
    SELECT CustomerId, EmployeeId
    FROM Sales.[Order]
    WHERE OrderDate >= '2016-01-01' AND OrderDate < '2016-02-01'
),
FebruaryOrders AS (
    SELECT CustomerId, EmployeeId
    FROM Sales.[Order]
    WHERE OrderDate >= '2016-02-01' AND OrderDate < '2016-03-01'
)
SELECT j.CustomerId, j.EmployeeId
FROM JanuaryOrders j
LEFT JOIN FebruaryOrders f ON j.CustomerId = f.CustomerId AND j.EmployeeId = f.EmployeeId
WHERE f.CustomerId IS NULL AND f.EmployeeId IS NULL
ORDER BY CustomerId;


# The EXCEPT Operator

## My Answer
**Proposition:** Write a query to select all unique combinations of country, region, and city for employees that do not match any combination of country, region, and city for customers.

**Tables:** HumanResources.Employee and Sales.Customer

**Columns:**
EmployeeCountry, EmployeeRegion, EmployeeCity from HumanResources.Employee and CustomerCountry, CustomerRegion, CustomerCity from Sales.Customer

**Predicate:** 

```
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.Employee
EXCEPT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.Customer;
```

The query uses the EXCEPT operator to return all distinct rows from the first SELECT statement that are not found in the second SELECT statement.

**What's special:**

**Set Operation (EXCEPT):** 
Identifies areas where the company has employees but does not have customers.


In [ ]:
USE TSQLV6;
-- Employees EXCEPT Customers
SELECT country, region, city FROM HR.Employees
EXCEPT
SELECT country, region, city FROM Sales.Customers;


USE Northwinds2022TSQLV7;
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.Employee
EXCEPT
SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.Customer;

# Circumventing Unsupported Logical Phases (Optional, Advanced)

**Proposition:** Write a query that calculates the number of distinct locations (combining region and city) for both employees and customers within each country.

**Tables:** HumanResources.Employee and Sales.Customer

**Columns:** EmployeeCountry, EmployeeRegion, EmployeeCity from HumanResources.Employee and CustomerCountry, CustomerRegion, CustomerCity from Sales.Customer

**Predicate:** The query uses the UNION operator to combine and de-duplicate location information from both employees and customers across all countries.

**What's special:**

**UNION Operator:** 
```
SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.Employee
      UNION
      SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.Customer
```

Merges two sets of location data: one from the HumanResources.Employee table and the other from the Sales.Customer table. The UNION operator removes duplicates, ensuring that each combination of country, region, and city is counted only once, regardless of whether it's listed for an employee or a customer.

**Derived Table (Subquery):** 

```
(SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.Employee
      UNION
      SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.Customer) AS U
```

The combined results are treated as a derived table (aliased as U), from which the query selects EmployeeCountry (or CustomerCountry, since they are unified in the derived table) and a count of distinct locations in each country.

**GROUP BY Clause:** This groups the results by country, allowing for the count of unique locations within each country.

**COUNT(*) AS NumLocations (Aggregation):** This counts the number of distinct location entries (region and city combinations) within each country.


In [ ]:
USE TSQLV6;
-- Number of distinct locations
-- that are either employee or customer locations in each country
SELECT country, COUNT(*) AS numlocations
FROM (SELECT country, region, city FROM HR.Employees
      UNION
      SELECT country, region, city FROM Sales.Customers) AS U
GROUP BY country;


USE Northwinds2022TSQLV7;
-- Number of distinct locations
-- that are either employee or customer locations in each country
SELECT EmployeeCountry, COUNT(*) AS NumLocations
FROM (SELECT EmployeeCountry, EmployeeRegion, EmployeeCity FROM HumanResources.Employee
      UNION
      SELECT CustomerCountry, CustomerRegion, CustomerCity FROM Sales.Customer) AS U
GROUP BY EmployeeCountry;